In [1]:
import sys
path_append = "./"
sys.path.append(path_append)  # Go up one directory from where you are.

import requests
import pandas as pd
import numpy as np
from datetime import datetime

from DataPreprocessing.Preprocessing import DataPreprocessor

from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.model_selection import  train_test_split
from ccnets.config import get_parser
from ccnets.ccnets import CCNets
from ccnets.resnets import ResNets
from nn.custom_deepfm import DeepFM
from nn.custom_dnn import ResMLP, MLP 
from ccnets.utils.log import create_log_details, create_log_name
from ccnets.utils.setting import set_random_seed
import torch
import os
from torch.utils.tensorboard import SummaryWriter

# Load Data

In [2]:
# # 한국마사회_경주성적정보 API
# # https://www.data.go.kr/iim/api/selectAPIAcountView.do 


# api_key = "mOX2kpzH42CzIPKdgYi75HQ6pkdETp%2BWe%2FS30VoA2VMdppX5dP21a1mOYW32hWJH1u0nF3%2BDc6Tkh8%2FBn4HBlw%3D%3D"  # 발급받은 실제 API 키로 변경하세요.
# url = f"https://apis.data.go.kr/B551015/API214_1/RaceDetailResult_1?serviceKey={api_key}&pageNo=1&numOfRows={10000000}&meet=3&rc_year={2023}&rc_month=&rc_date=&rc_no=&_type=json"
# # https://apis.data.go.kr/B551015/API214_1/RaceDetailResult_1?
# # serviceKey=mOX2kpzH42CzIPKdgYi75HQ6pkdETp%2BWe%2FS30VoA2VMdppX5dP21a1mOYW32hWJH1u0nF3%2BDc6Tkh8%2FBn4HBlw%3D%3D
# # &pageNo=1&numOfRows=10&meet=1&rc_year=2022&rc_month=202202&rc_date=20220220&rc_no=1&_type=xml


# response = requests.get(url)

# # 응답 상태 코드가 200(정상)인 경우에만 내용을 출력합니다.
# if response.status_code == 200:
##    print(response.text)
# else:
#     print("Error:", response.status_code)

# data = response.json()
## print(data)
# items = data['response']['body']['items']['item']
# df = pd.json_normalize(items)

In [3]:
## df save
# df.to_csv('./horseRaceBusan.csv')

## df load
df = pd.read_csv('./horseRaceBusan.csv') 

In [4]:
df.tail()

,Unnamed: 0,age,ageCond,birthday,buG1fAccTime,buG1fOrd,buG2fAccTime,buG2fOrd,buG3fAccTime,buG3fOrd,...,sj_4cOrd,trName,trNo,track,wgBudam,wgBudamBigo,wgHr,wgJk,winOdds,weather
5321,5321,3,연령오픈,20200321,89.3,6,76.0,6,63.0,5,...,0,김병학,70121,건조 (2%),54.5,-,482(),0,2.4,맑음
5322,5322,6,연령오픈,20170315,89.2,4,75.9,5,63.2,6,...,0,장세한,70127,건조 (2%),52.0,-,441(+3),0,131.5,맑음
5323,5323,4,연령오픈,20190505,89.4,9,75.8,4,62.6,3,...,0,토마스,70194,건조 (2%),54.0,-,519(-1),0,16.6,맑음
5324,5324,4,연령오픈,20190320,89.7,11,76.5,8,63.7,8,...,0,백광열,70130,건조 (2%),54.0,-,507(-11),0,22.4,맑음
5325,5325,5,연령오픈,20180510,90.5,12,77.3,12,64.4,11,...,0,윤영훈,70133,건조 (2%),52.0,-,455(+1),0,103.5,맑음


In [5]:
df.columns

Index(['Unnamed: 0', 'age', 'ageCond', 'birthday', 'buG1fAccTime', 'buG1fOrd',
       'buG2fAccTime', 'buG2fOrd', 'buG3fAccTime', 'buG3fOrd', 'buG4fAccTime',
       'buG4fOrd', 'buG6fAccTime', 'buG6fOrd', 'buG8fAccTime', 'buG8fOrd',
       'buS1fAccTime', 'buS1fOrd', 'buS1fTime', 'bu_10_8fTime', 'bu_1fGTime',
       'bu_2fGTime', 'bu_3fGTime', 'bu_4_2fTime', 'bu_6_4fTime', 'bu_8_6fTime',
       'budam', 'buga1', 'buga2', 'buga3', 'chaksun1', 'chaksun2', 'chaksun3',
       'chaksun4', 'chaksun5', 'chulNo', 'diffUnit', 'hrName', 'hrNo',
       'hrTool', 'ilsu', 'jeG1fTime', 'jeG3fTime', 'jeS1fTime', 'je_1cTime',
       'je_2cTime', 'je_3cTime', 'je_4cTime', 'jkName', 'jkNo', 'meet', 'name',
       'ord', 'ordBigo', 'owName', 'owNo', 'plcOdds', 'prizeCond', 'rank',
       'rankRise', 'rating', 'rcDate', 'rcDay', 'rcDist', 'rcName', 'rcNo',
       'rcTime', 'seG1fAccTime', 'seG3fAccTime', 'seS1fAccTime',
       'se_1cAccTime', 'se_2cAccTime', 'se_3cAccTime', 'se_4cAccTime', 'sex',
       '

# Preprocessing

In [6]:
df = df[['age', 'ageCond', 'birthday', 'buG1fAccTime', 'buG1fOrd',
       'buG2fAccTime', 'buG2fOrd', 'buG3fAccTime', 'buG3fOrd', 'buG4fAccTime',
       'buG4fOrd', 'buG6fAccTime', 'buG6fOrd', 'buG8fAccTime', 'buG8fOrd',
       'buS1fAccTime', 'buS1fOrd', 'buS1fTime', 'bu_10_8fTime', 'bu_1fGTime',
       'bu_2fGTime', 'bu_3fGTime', 'bu_4_2fTime', 'bu_6_4fTime', 'bu_8_6fTime',
       'budam', 'chulNo', 'diffUnit', 'hrName', 'hrNo',
       'hrTool', 'ilsu', 'jkName', 'jkNo', 'meet', 'name',
       'ord', 'ordBigo', 'owName', 'owNo', 'plcOdds', 'prizeCond', 'rank',
       'rankRise', 'rating', 'rcDate', 'rcDay', 'rcDist', 'rcName', 'rcNo',
       'rcTime', 'sex', 'trName', 'trNo', 'track', 'wgBudam', 'wgBudamBigo', 'wgHr',
       'wgJk', 'winOdds', 'weather']]

In [7]:
df.head()

,age,ageCond,birthday,buG1fAccTime,buG1fOrd,buG2fAccTime,buG2fOrd,buG3fAccTime,buG3fOrd,buG4fAccTime,...,sex,trName,trNo,track,wgBudam,wgBudamBigo,wgHr,wgJk,winOdds,weather
0,4,연령오픈,20190227,0.0,0,0.0,0,0.0,0,0.0,...,암,박재호,70226,(0%),54.0,-,0(),0,0.0,NaN
1,2,연령오픈,20210310,0.0,0,0.0,0,0.0,0,0.0,...,암,강형곤,70134,(0%),52.5,-,0(),0,0.0,NaN
2,2,연령오픈,20210420,0.0,0,0.0,0,0.0,0,0.0,...,암,강은석,70223,(0%),52.5,-,0(),0,0.0,NaN
3,2,연령오픈,20210123,0.0,0,0.0,0,0.0,0,0.0,...,암,울즐리,70156,(0%),52.5,-,0(),0,0.0,NaN
4,2,연령오픈,20210319,0.0,0,0.0,0,0.0,0,0.0,...,암,조용배,70137,(0%),52.5,-,0(),0,0.0,NaN


In [8]:
df.columns

Index(['age', 'ageCond', 'birthday', 'buG1fAccTime', 'buG1fOrd',
       'buG2fAccTime', 'buG2fOrd', 'buG3fAccTime', 'buG3fOrd', 'buG4fAccTime',
       'buG4fOrd', 'buG6fAccTime', 'buG6fOrd', 'buG8fAccTime', 'buG8fOrd',
       'buS1fAccTime', 'buS1fOrd', 'buS1fTime', 'bu_10_8fTime', 'bu_1fGTime',
       'bu_2fGTime', 'bu_3fGTime', 'bu_4_2fTime', 'bu_6_4fTime', 'bu_8_6fTime',
       'budam', 'chulNo', 'diffUnit', 'hrName', 'hrNo', 'hrTool', 'ilsu',
       'jkName', 'jkNo', 'meet', 'name', 'ord', 'ordBigo', 'owName', 'owNo',
       'plcOdds', 'prizeCond', 'rank', 'rankRise', 'rating', 'rcDate', 'rcDay',
       'rcDist', 'rcName', 'rcNo', 'rcTime', 'sex', 'trName', 'trNo', 'track',
       'wgBudam', 'wgBudamBigo', 'wgHr', 'wgJk', 'winOdds', 'weather'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 61 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   age           5326 non-null   int64  
 1   ageCond       5326 non-null   object 
 2   birthday      5326 non-null   int64  
 3   buG1fAccTime  5326 non-null   float64
 4   buG1fOrd      5326 non-null   int64  
 5   buG2fAccTime  5326 non-null   float64
 6   buG2fOrd      5326 non-null   int64  
 7   buG3fAccTime  5326 non-null   float64
 8   buG3fOrd      5326 non-null   int64  
 9   buG4fAccTime  5326 non-null   float64
 10  buG4fOrd      5326 non-null   int64  
 11  buG6fAccTime  5326 non-null   float64
 12  buG6fOrd      5326 non-null   int64  
 13  buG8fAccTime  5326 non-null   float64
 14  buG8fOrd      5326 non-null   int64  
 15  buS1fAccTime  5326 non-null   float64
 16  buS1fOrd      5326 non-null   int64  
 17  buS1fTime     5326 non-null   float64
 18  bu_10_8fTime  5326 non-null 

In [10]:
edf = df[['age', 'ageCond', 'birthday',
       'budam', 'chulNo', 'hrTool', 'jkName', 'name', 'owName',
       'prizeCond', 'rank', 'rating', 'rcDist', 'rcName', 'rcNo', 'rcTime', 'sex', 'trName', 'track',
       'wgBudam', 'wgHr', 'wgJk', 'weather']]

In [11]:
edf.isnull().sum()

age           0
ageCond       0
birthday      0
budam         0
chulNo        0
hrTool        0
jkName        0
name          0
owName        0
prizeCond     0
rank          0
rating        0
rcDist        0
rcName        0
rcNo          0
rcTime        0
sex           0
trName        0
track         0
wgBudam       0
wgHr          0
wgJk          0
weather      68
dtype: int64

In [12]:
# 전처리기 정의
preprocessor = DataPreprocessor(edf)
df = preprocessor.fillna_zero()

c:\Users\jhp\Desktop\ccnets-api-main\DataPreprocessing\Preprocessing.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.fillna(0, inplace=True)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        5326 non-null   int64  
 1   ageCond    5326 non-null   object 
 2   birthday   5326 non-null   int64  
 3   budam      5326 non-null   object 
 4   chulNo     5326 non-null   int64  
 5   hrTool     5326 non-null   object 
 6   jkName     5326 non-null   object 
 7   name       5326 non-null   object 
 8   owName     5326 non-null   object 
 9   prizeCond  5326 non-null   object 
 10  rank       5326 non-null   object 
 11  rating     5326 non-null   int64  
 12  rcDist     5326 non-null   int64  
 13  rcName     5326 non-null   object 
 14  rcNo       5326 non-null   int64  
 15  rcTime     5326 non-null   float64
 16  sex        5326 non-null   object 
 17  trName     5326 non-null   object 
 18  track      5326 non-null   object 
 19  wgBudam    5326 non-null   float64
 20  wgHr    

In [14]:
# age컬럼 범주형으로 변경
df['age'] = df['age'].astype(str)

In [15]:
import pandas as pd

# 'birthday' 컬럼을 날짜 형식으로 변환
df['birthday'] = pd.to_datetime(df['birthday'], format='%Y%m%d')

# 현재까지의 일수 계산
df['days_since_birth'] = (pd.Timestamp.now() - df['birthday']).dt.days

# birthday 컬럼 제거
df.drop('birthday', axis = 1, inplace = True)

In [16]:
df.columns

Index(['age', 'ageCond', 'budam', 'chulNo', 'hrTool', 'jkName', 'name',
       'owName', 'prizeCond', 'rank', 'rating', 'rcDist', 'rcName', 'rcNo',
       'rcTime', 'sex', 'trName', 'track', 'wgBudam', 'wgHr', 'wgJk',
       'weather', 'days_since_birth'],
      dtype='object')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               5326 non-null   object 
 1   ageCond           5326 non-null   object 
 2   budam             5326 non-null   object 
 3   chulNo            5326 non-null   int64  
 4   hrTool            5326 non-null   object 
 5   jkName            5326 non-null   object 
 6   name              5326 non-null   object 
 7   owName            5326 non-null   object 
 8   prizeCond         5326 non-null   object 
 9   rank              5326 non-null   object 
 10  rating            5326 non-null   int64  
 11  rcDist            5326 non-null   int64  
 12  rcName            5326 non-null   object 
 13  rcNo              5326 non-null   int64  
 14  rcTime            5326 non-null   float64
 15  sex               5326 non-null   object 
 16  trName            5326 non-null   object 


In [18]:
int_columns = df.select_dtypes(include=['int']).columns
print(int_columns)


Index(['chulNo', 'rating', 'rcDist', 'rcNo', 'wgJk', 'days_since_birth'], dtype='object')


In [19]:
# df[df['track'] == ' (0%)']['track'] = '건조'
df.loc[df['track'] == ' (0%)', 'track'] = '건조'


In [20]:
df['track'] = df['track'].str.split(' ').str[0]

In [21]:
df

,age,ageCond,budam,chulNo,hrTool,jkName,name,owName,prizeCond,rank,...,rcNo,rcTime,sex,trName,track,wgBudam,wgHr,wgJk,weather,days_since_birth
0,4,연령오픈,별정A,9,"계란형큰,망사눈",전진구,한국,강영종,R0~0,국6등급,...,1,0.0,암,박재호,건조,54.0,0(),0,0,1650
1,2,연령오픈,별정A,6,"Triabit ,망사",박재이,한국,함승원,R0~0,국6등급,...,1,0.0,암,강형곤,건조,52.5,0(),0,0,908
2,2,연령오픈,별정A,7,"계란형큰,망사,승인편자,혀끈+",채상현,한국,현길림,R0~0,국6등급,...,1,0.0,암,강은석,건조,52.5,0(),0,0,867
3,2,연령오픈,별정A,8,"JR러깅,망사",모준호,한국,이시돌협회,R0~0,국6등급,...,1,0.0,암,울즐리,건조,52.5,0(),0,0,954
4,2,연령오픈,별정A,12,"Triabit ,망사+",김동영,한국,강용익,R0~0,국6등급,...,1,0.0,암,조용배,건조,52.5,0(),0,0,899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5321,3,연령오픈,별정B,2,계란형큰,다실바,한국,신윤경,R1~50,국4등급,...,10,103.6,수,김병학,건조,54.5,482(),0,맑음,1262
5322,6,연령오픈,별정B,4,"계란형큰,망사",전진구,한국,조창환,R1~50,국4등급,...,10,104.0,암,장세한,건조,52.0,441(+3),0,맑음,2364
5323,4,연령오픈,별정B,5,"망사눈,반가지큰",김어수,한국,정영식,R1~50,국4등급,...,10,104.0,암,토마스,건조,54.0,519(-1),0,맑음,1583
5324,4,연령오픈,별정B,1,"계란형큰,망사눈",최시대,한국,이장현,R1~50,국4등급,...,10,104.4,거,백광열,건조,54.0,507(-11),0,맑음,1629


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5326 entries, 0 to 5325
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               5326 non-null   object 
 1   ageCond           5326 non-null   object 
 2   budam             5326 non-null   object 
 3   chulNo            5326 non-null   int64  
 4   hrTool            5326 non-null   object 
 5   jkName            5326 non-null   object 
 6   name              5326 non-null   object 
 7   owName            5326 non-null   object 
 8   prizeCond         5326 non-null   object 
 9   rank              5326 non-null   object 
 10  rating            5326 non-null   int64  
 11  rcDist            5326 non-null   int64  
 12  rcName            5326 non-null   object 
 13  rcNo              5326 non-null   int64  
 14  rcTime            5326 non-null   float64
 15  sex               5326 non-null   object 
 16  trName            5326 non-null   object 


In [23]:
df['wgHr'] = df['wgHr'].str.split('(').str[1].str.split(')').str[0]
df.loc[df['wgHr'] == '', 'wgHr'] = 0
df['wgHr'] = df['wgHr'].astype(int)

In [24]:
target = 'rcTime'

X = df.drop(target, axis = 1)
y = df[target]

In [25]:
# DataPreprocessor 클래스의 인스턴스 생성
preprocessor = DataPreprocessor(X)

# One-Hot Encoding 수행
X_encoded = preprocessor.one_hot_encode()

# Robust Scaling 수행
X = preprocessor.robust_scale()

# 결과 확인
print(X)

      chulNo  rating  rcDist  rcNo   wgBudam  wgHr  wgJk  days_since_birth  \
0          9       0    1200     1  0.000000     0     0              1650   
1          6       0    1200     1 -0.500000     0     0               908   
2          7       0    1200     1 -0.500000     0     0               867   
3          8       0    1200     1 -0.500000     0     0               954   
4         12       0    1200     1 -0.500000     0     0               899   
...      ...     ...     ...   ...       ...   ...   ...               ...   
5321       2      40    1600    10  0.166667     0     0              1262   
5322       4      36    1600    10 -0.666667     3     0              2364   
5323       5      42    1600    10  0.000000    -1     0              1583   
5324       1      38    1600    10  0.000000   -11     0              1629   
5325       7      37    1600    10 -0.666667     1     0              1943   

      age_10  age_2  ...  trName_홍대유  track_건조  track_다습  track

In [26]:
y

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
        ...  
5321    103.6
5322    104.0
5323    104.0
5324    104.4
5325    104.9
Name: rcTime, Length: 5326, dtype: float64

In [27]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        self.x = np.array(self.x).astype(float)
        vals = torch.tensor(self.x[index], dtype = torch.float32)
        label = torch.tensor(self.y[index], dtype= torch.float32).unsqueeze(-1)
        return vals, label


In [28]:
args = get_parser()
args.device = torch.device('cuda:0' if (torch.cuda.is_available() and args.ngpu > 0) else "cpu")

In [29]:
import IPython ; file_path = IPython.extract_module_locals()[1]['__vsc_ipynb_file__']
from pathlib import Path
file_name = Path(file_path).stem
model_path = path_append + f"/models/{file_name}/"
temp_path = path_append + f"/models/{'temp_'}{file_name}/"
log_path = path_append + f"/log/{file_name}/"


if Path(temp_path).exists() is False: 
    os.mkdir(temp_path)

if Path(model_path).exists() is False: 
    os.mkdir(model_path)

if Path(log_path).exists() is False: 
    os.mkdir(log_path)  

args.model_path = model_path
args.temp_path = temp_path

In [30]:
args.num_epoch = 1000
args.lr = 2e-4
args.batch_size = 256
args.step_size = 10

args.num_layer = 4
args.hidden_size = 128

args.obs_size = 836
args.label_size = 1
args.explain_size = 32
args.seq_len = 0

args.num_checkpoints = 100
args.use_one_hot = False

args.reasoner_joint_type = "add"
args.producer_joint_type = "add"
args.label_type = "R" 

args.obs_fn = "none"
args.label_fn = "none"
test_size = 0.2

X = X[:]
# args.num_epoch = int(round(3.2/(1 - test_size)))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, shuffle= False)


X_train = X_train.iloc[:, :].values 
X_test = X_test.iloc[:, :].values 
y_train = y_train.values
y_test = y_test.values

trainset = Dataset(X_train, y_train)
testset = Dataset(X_test, y_test)


In [31]:
print(X.shape)
print(y.shape)

(5326, 836)
(5326,)


In [37]:
import torch.nn as nn

class SuperNetBlock(nn.Module):
    def __init__(self, args):
        super(SuperNetBlock, self).__init__()
        # Different possible operations within the block
        self.linear1 = nn.Linear(hidden_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        # Architecture weights for each operation
        self.arch_weights = nn.Parameter(torch.ones(2) / 2)  # Initialize close to a uniform distribution
    def forward(self, x):
        weights = F.softmax(self.arch_weights, dim=-1)
        out = weights[0] * self.linear1(x) + weights[1] * self.linear2(x)
        return out
class SuperNet(nn.Module):
    def __init__(self, args):
        self.num_layer = args.num_layer
        hidden_dim = args.hidden_size
        layers = []
        for i in range(self.num_layer):
            layers.append(SuperNetBlock(hidden_dim))
            layers.append(nn.ReLU())
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

In [38]:

args.loss_type = "L1"
args.error_type = "L1"
args.loss_reduction = "all"
args.error_reduction = "none"
log_details = create_log_details(args)
args.log = SummaryWriter(log_dir=create_log_name(log_path, log_details))
set_random_seed(0)

ccnets = CCNets(args, SuperNet, SuperNet, SuperNet)
ccnets.train(trainset, testset)

# args.loss_type = "MSE"
# args.loss_reduction = "all"
# log_details = create_log_details(args)
# args.log = SummaryWriter(log_dir=create_log_name(log_path, log_details))
# set_random_seed(0)
# resnets = ResNets(args, MLP, DeepFM)
# resnets.train(trainset, testset)

TypeError: __init__() missing 1 required positional argument: 'hidden_dim'